In [2]:
import networkx as nx
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

In [71]:
df = pd.read_csv("data/wtf.csv")
df['sitc4'] = df['sitc4'].astype(str)
del df['Unnamed: 0']

gdf = df.groupby(["sitc4","exporter"]).sum().reset_index()
gdf = gdf[['sitc4','exporter','value']]

rca_df = pd.read_csv("data/rca.csv")
del rca_df['Unnamed: 0']
rca_df = rca_df.dropna(subset=['sitc4', 'exporter'])


/Users/francisco.villalobos/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.



# $M_{cp}$

In [98]:
rca_treshold = 1
mcp = rca_df[rca_df.rca >= rca_treshold]
mcp

,sitc4,exporter,value,rca
1,0015,Areas NES,39012,23.301008
2,001X,Areas NES,30,54.200398
5,0141,Areas NES,158,1.791831
11,02XX,Areas NES,102,111.224206
18,0412,Areas NES,38224,4.208621
...,...,...,...,...
62398,7449,Eur.Other NE,496,9.559791
62399,7919,Eur.Other NE,2110,143.167907
62400,5416,CACM NES,518,553.623273
62401,6841,Br.Antr.Terr,2262,251.251165


#  $k_{c,0} = \sum_{p} M_{cp}$

In [99]:
country = "Germany"
mdf = mcp[mcp.exporter==country]
len(mdf)

473

#  $k_{p,0} = \sum_{c} M_{cp}$

In [100]:
product = "0011"
mdf = mcp[mcp.sitc4==product]
len(mdf)


20

# Initialize k reflection dictionary

In [101]:
k = dict()
for product in mcp.sitc4.unique():
    mdf = mcp[mcp.sitc4==product]
    k[product]=[len(mdf)]
for country in mcp.exporter.unique():
    mdf = mcp[mcp.exporter==country]
    k[country]=[len(mdf)]
    

#  $k_{c,1} = \frac{1}{k_{c,0}} \sum_{p} M_{cp}k_{p,0}$

### Test on Germany

In [102]:
country = "Germany"
mdf = mcp[mcp.exporter==country]
sum_kp = 0
for product in mdf.sitc4.unique():
    sum_kp += k[product][0]

sum_kp/k[country][0]

18.101479915433405

### First iteration countries

In [103]:
for country in mcp.exporter.unique():
    mdf = mcp[mcp.exporter==country]
    sum_kp = 0
    for product in mdf.sitc4.unique():
        sum_kp += k[product][0]
    k[country].append(sum_kp/k[country][0])

In [104]:
k["USA"]

[406, 18.194581280788178]

#  $k_{p,1} = \frac{1}{k_{p,0}} \sum_{p} M_{cp}k_{c,0}$

### Test sitc4 0011

In [105]:
product = "0015"
mdf = mcp[mcp.sitc4==product]
sum_kc = 0
for country in mdf.exporter.unique():
    sum_kc += k[country][0]
sum_kc/k[product][0]

320.2631578947368

### All products

In [106]:
for product in mcp.sitc4.unique():
    mdf = mcp[mcp.sitc4==product]
    sum_kc = 0
    for country in mdf.exporter.unique():
        sum_kc += k[country][0]
    k[product].append(sum_kc/k[product][0])

In [107]:
country_list = list(mcp.exporter.unique())

In [108]:
country_list

['Areas NES',
 'Argentina',
 'Australia',
 'Austria',
 'Belgium-Lux',
 'Belize',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Canada',
 'China',
 'Czech Rep',
 'Denmark',
 'Ecuador',
 'Egypt',
 'Estonia',
 'France,Monac',
 'Germany',
 'Greece',
 'Honduras',
 'Hungary',
 'Ireland',
 'Italy',
 'Jordan',
 'Lao P.Dem.R',
 'Lithuania',
 'Mexico',
 'Myanmar',
 'Netherlands',
 'New Zealand',
 'Nicaragua',
 'Norway',
 'Pakistan',
 'Panama',
 'Paraguay',
 'Poland',
 'Portugal',
 'Romania',
 'Russian Fed',
 'Singapore',
 'Slovakia',
 'South Africa',
 'Spain',
 'Sweden',
 'Thailand',
 'UK',
 'USA',
 'Untd Arab Em',
 'Uruguay',
 'World',
 'Djibouti',
 'Ethiopia',
 'Iran',
 'Lebanon',
 'Oman',
 'Qatar',
 'Saudi Arabia',
 'Somalia',
 'Sudan',
 'Syria',
 'Turkey',
 'Uganda',
 'Yemen',
 'Algeria',
 'Chile',
 'Cyprus',
 'Finland',
 'Indonesia',
 'Korea Rep.',
 'Mali',
 'Colombia',
 'Israel',
 'Japan',
 'Malaysia',
 'Peru',
 'Venezuela',
 'Belarus',
 'Bosnia Herzg',
 'China HK SAR',
 'Costa Rica',
 'Croatia',

# phase plot

In [109]:
country_list = ['Estonia',
 'France,Monac',
 'Germany',
 'Greece',
 'Honduras']

country_list = list(mcp.exporter.unique())
country_list.remove("World")


kdf = pd.DataFrame({'country_name': country_list,
                    'kc0': list(map(lambda x: k[x][0],country_list)),
                   'kc1': list(map(lambda x: k[x][1],country_list))})

layout = dict(plot_bgcolor='white', xaxis=dict(title='$k_{c,0}$', linecolor='#d9d9d9', mirror=True),
              yaxis=dict(title='$k_{c,1}$', linecolor='#d9d9d9', mirror=True))

data = go.Scatter(x=kdf['kc0'],
                  y=kdf['kc1'],
                  text=kdf['country_name'],
                  mode='markers+text',
                  marker=dict(color='rgba(48, 217, 189, 1)'),
                  name='citations')

fig = go.Figure(data=data, layout=layout)

fig.show()